Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Fake News Predictor


##Data

In [ ]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
#Read the data
df=pd.read_csv('/content/drive/MyDrive/Study/Fake News data/news.csv')

In [ ]:
#Get shape and head
df.shape
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
df.label.size

6335

In [ ]:
df.label

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [ ]:
#Get the labels
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

## PAC

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=train_test_split(df['text'], labels, test_size=0.2, random_state=7)

In [ ]:
#Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

In [ ]:
#Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [ ]:
#Initialize a PassiveAggressiveClassifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
use_idf=True

In [ ]:
#Predict on the train set and calculate accuracy
y_pred_train=pac.predict(tfidf_train)
score=accuracy_score(y_train,y_pred_train)
print(f'Training Accuracy: {round(score*100,2)}%')

Training Accuracy: 100.0%


In [ ]:
#Build confusion matrix
confusion_matrix(y_train,y_pred_train, labels=['FAKE','REAL'])

array([[2526,    0],
       [   0, 2542]])

In [ ]:
#Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Testing Accuracy: {round(score*100,2)}%')

Testing Accuracy: 92.74%


In [ ]:
#Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

array([[588,  50],
       [ 42, 587]])

In [ ]:
def predictor(newsline):
  data =[newsline]
  df3= pd.DataFrame(data, columns = ['Text']) 
  nl = tfidf_vectorizer.transform(df3)
  prediction = pac.predict(nl)
  return prediction[0]

In [ ]:
newsline="Donald Trump killed Messi in a Football Match"
predictor(newsline)

In [ ]:
newsline="Obama was a US President"
predictor(newsline)

In [ ]:
predictor(df.iloc[4].text)

Suggestions

In [ ]:
tfidf, naive baise

In [ ]:
Dataset  dakshit google 

In [ ]:
stop words

## LSTM

In [ ]:
df['labels'] = np.where(df['label'] == "REAL", 1, 0)

In [ ]:
df

,Unnamed: 0,title,text,label,labels
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1
...,...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL,1


In [ ]:
labels=df.labels
labels.head()

0    0
1    0
2    1
3    0
4    1
Name: labels, dtype: int64

In [ ]:
def input_padded_oh(docs, max_length):

  # integer encode the documents
  encoded_docs = [one_hot(d, vocab_size) for d in docs]
  
  # pad documents to a max length of 4 words
  padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
  
  return padded_docs

In [ ]:
max_words = 250
vocab_size = 2500

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],labels,test_size=0.3)

In [ ]:
#X_train = pad_sequences(X_train, maxlen=max_words)
X_train = input_padded_oh(X_train, max_words)

In [ ]:
#X_test = pad_sequences(X_test, maxlen=max_words)
X_test = input_padded_oh(X_test, max_words)

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 32)           80000     
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 133,301
Trainable params: 133,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
metrics = tf.keras.metrics.BinaryAccuracy()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
# compile the model
model.compile(optimizer='adam', loss=loss, metrics=metrics)

In [ ]:
labels.dtypes

dtype('int64')

In [ ]:
x_test.shape

(1267,)

In [ ]:
# fit the model
#model.fit(X_train, y_train, epochs=500, verbose=0)
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


69/69 [==============================] - 20s 257ms/step - loss: 0.6629 - binary_accuracy: 0.6275 - val_loss: 0.6406 - val_binary_accuracy: 0.7031
Epoch 2/3
69/69 [==============================] - 17s 249ms/step - loss: 0.5474 - binary_accuracy: 0.7492 - val_loss: 0.5537 - val_binary_accuracy: 0.7031
Epoch 3/3
69/69 [==============================] - 17s 249ms/step - loss: 0.3978 - binary_accuracy: 0.8243 - val_loss: 0.5370 - val_binary_accuracy: 0.6719


In [ ]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Test accuracy: 0.7648605704307556


In [ ]:
df.iloc[4].text

'It\'s primary day in New York and front-runners Hillary Clinton and Donald Trump are leading in the polls.\n\nTrump is now vowing to win enough delegates to clinch the Republican nomination and prevent a contested convention. But Sens.Ted Cruz, R-Texas, Bernie Sanders, D-Vt., and Ohio Gov. John Kasich and aren\'t giving up just yet.\n\nA big win in New York could tip the scales for both the Republican and Democratic front-runners in this year\'s race for the White House. Clinton and Trump have each suffered losses in recent contests, shifting the momentum to their rivals.\n\n"We have won eight out of the last nine caucuses and primaries! Cheer!" Sanders recently told supporters.\n\nWhile wins in New York for Trump and Clinton are expected, the margins of those victories are also important.\n\nTrump needs to capture more than 50 percent of the vote statewide if he wants to be positioned to win all of the state\'s 95 GOP delegates. That would put him one step closer to avoiding a contes

In [ ]:
# Test1
line=df.iloc[4].text
x= one_hot(line, vocab_size)
x= pad_sequences([x], maxlen=max_words, padding='post')
print(model.predict(x))

[[0.7575027]]


In [ ]:
# Test2
line=df.iloc[3].text
x= one_hot(line, vocab_size)
x= pad_sequences([x], maxlen=max_words, padding='post')
print(model.predict(x))

[[0.12322134]]


# Keyword Extraction

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#use this to scrap data from web
scraped_data = urllib.request.urlopen('https://www.indiatravelblog.com/resources/6138-a-day-trip-to-chittorgarh-from-udaipur')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

In [ ]:
article_text

"\nAre you visiting Udaipur in Rajasthan? Plan a one day trip to Chittorgarh Fort, when deciding on an itinerary. The Chittorgarh Fort is the biggest fort in India, with magnificent architecture and vibrant and exciting history, of valiant kings, battles and sieges, and tales of romance and Jauhar.\nI was in Udaipur, for a friend's son's wedding, where I met up with a bunch of old friends. We extended our stay by a few days, and spent the time, exploring places of interest around this beautiful city. It was the time when the movie Padmavat was in the news, hogging primetime debates on national television, and eliciting quite a controversy. Curiosity got the better of us, and we couldn't resist the urge to visit Chittorgarh, the famed site where history was created. We set off early, seated comfortably in a Tempo Traveller, booked the previous evening. Chittorgarh is roughly 115 km from Udaipur, and it takes just over 2 ½ hours to reach. It's a comfortable stretch on well-laid National 

In [ ]:
def preprocessmyblog(text):
    text=re.sub(r'\d+', '', text)#remove number
    text=text.lower()#lower case
    translator = str.maketrans('', '', string.punctuation)
    text=text.translate(translator)#remove punc
    text=" ".join(text.split())#extra space
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    text = [word for word in word_tokens if word not in stop_words] #stop words removal
    #text = [stemmer.stem(word) for word in text] #Stemming
    #lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text] #Lemmatization
    return text

In [ ]:
text = preprocessmyblog(article_text)
text= " ".join(text)

In [ ]:
text

'visiting udaipur rajasthan plan one day trip chittorgarh fort deciding itinerary chittorgarh fort biggest fort india magnificent architecture vibrant exciting history valiant kings battles sieges tales romance jauhar udaipur friends sons wedding met bunch old friends extended stay days spent time exploring places interest around beautiful city time movie padmavat news hogging primetime debates national television eliciting quite controversy curiosity got better us couldnt resist urge visit chittorgarh famed site history created set early seated comfortably tempo traveller booked previous evening chittorgarh roughly km udaipur takes ½ hours reach comfortable stretch welllaid national highways mention good enough road trip bike car public transport two cities good adsbygoogle windowadsbygoogle push drivers assistant kuldip acted unofficial tour guide kuldip qualified part born brought parts many tale romance bravery share course none could authenticated constant jabber kept us engaged p

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
##Creating a list of custom stopwords
#new_words = ["definig the words you can stop"]
#stop_words = list(stop_words.union(new_words))

def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    # remove stopwords
    text = [word for word in text if word not in stop_words]

    # remove words less than three letters
    text = [word for word in text if len(word) >= 3]

    # lemmatize
    lmtzr = WordNetLemmatizer()
    text = [lmtzr.lemmatize(word) for word in text]
    
    return ' '.join(text)

In [ ]:
from keybert import KeyBERT

model = KeyBERT(model="distilbert-base-nli-mean-tokens")


  0%|          | 0.00/245M [00:00<?, ?B/s]

In [ ]:
def _keybert(blog):
  first=model.extract_keywords(
      blog,
      top_n=10,
      keyphrase_ngram_range=(1, 3),
      stop_words="english",
      use_mmr=True,                   #Maximal Marginal Relevance for the selection of keywords/keyphrases
      diversity=0.5                  #diversity: The diversity of the results between 0 and 1 if use_mmr is set to True
      )
  return first

In [ ]:
key=_keybert(text)

In [ ]:
for i in key:
  print(i,'\n')

('hogging primetime debates', 0.536) 

('friends sons wedding', 0.4314) 

('rajasthan luxury tents', 0.4187) 

('reservoir popular tourist', 0.3953) 

('talking parrots extolling', 0.2731) 

('summer tripthis exhaustive', 0.3728) 

('plan vacation winter', 0.405) 

('destruction legends queens', 0.3484) 

('forts old castles', 0.3371) 

('ballads folk songs', 0.3771) 

